# 3. Diffusion Model Training 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import random
import numpy as np
import json
import math

# Set the random seed (for replicability)
seed = 20777980
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

def determine_max_seq_len(data, max_length='max_length'):
    """Calculate the max sequence length dynamically if 'max_length' is used as argument."""
    if max_length == 'max_length':
        MAX_LENGTH = max(len(dp["tokens"]) for dp in data)
    else:
        MAX_LENGTH = max_length
    return MAX_LENGTH

def setup_device():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    return device

def save_model(model, filepath):
    torch.save(model.state_dict(), filepath)
    return model

def load_model(model, filepath):
    model.load_state_dict(torch.load(filepath))
    model.eval()
    device = setup_device()
    model.to(device)
    return model, device

def load_dataset(file_path):
    """Load the dataset from a JSON file."""
    with open(file_path, 'r') as file:
        dataset = [json.loads(line) for line in file]
    return dataset

def save_JSON(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f)
    return

def load_JSON(filename):
    """Load a JSON file."""
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

class CosineNoiseSchedule:
    def __init__(self, timesteps=1000, epsilon=1e-6, device=None):
        self.timesteps = timesteps
        self.epsilon = epsilon
        self.device = device
        
        # Create alphas using a cosine schedule
        self.alphas = torch.cos(torch.linspace(0, math.pi / 2, timesteps, device=device)) ** 2
        self.betas = 1.0 - self.alphas
        self.alpha_bar = torch.maximum(torch.cumprod(self.alphas, dim=0), torch.tensor(self.epsilon, device=self.alphas.device))

    def get_alpha(self, t):
        return self.alphas[t]

    def get_beta(self, t):
        return self.betas[t]

    def get_variance(self, t):
        return self.get_beta(t) * (1 - self.get_alpha(t))

    def get_alpha_bar(self, t):
        return self.alpha_bar[t]

class DiffusionModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_heads, num_timesteps, max_seq_len=5000, pretrained_embeddings=None):
        super(DiffusionModel, self).__init__()

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_timesteps = num_timesteps

        # Use pre-trained embeddings if available, otherwise initialize embeddings randomly
        if pretrained_embeddings is not None:
            pretrained_embeddings = torch.tensor(list(pretrained_embeddings.values()), dtype=torch.float32)
            if pretrained_embeddings.size(1) != embedding_dim:
                raise ValueError(
                    f"Pretrained embeddings size {pretrained_embeddings.size(1)} does not match the required embedding_dim {embedding_dim}."
                )
            self.embedding = nn.Parameter(pretrained_embeddings)
        else:
            self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Projection layer to map from embedding_dim to hidden_dim if needed
        self.projection = nn.Linear(embedding_dim, hidden_dim) if embedding_dim != hidden_dim else nn.Identity()

        # Layer normalization
        self.layer_norm = nn.LayerNorm(hidden_dim, eps=1e-6)

        # Transformer setup
        self.transformer = nn.Transformer(
            d_model=hidden_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            batch_first=False  # Standard transformer expects (seq_len, batch_size, hidden_dim)
        )
        
        # Fully connected output layer to map back to embedding_dim
        self.fc_out = nn.Linear(hidden_dim, embedding_dim)

    def forward(self, embeddings):
        # embeddings expected to have shape (batch_size, hidden_dim, seq_len)
        batch_size, hidden_dim, seq_len = embeddings.shape

        # If embedding_dim != hidden_dim, apply projection
        if self.embedding_dim != self.hidden_dim:
            embeddings = self.projection(embeddings)  # (batch_size, hidden_dim, seq_len)

        embeddings = embeddings.transpose(1, 2)
        
        # Apply layer normalization
        embeddings = self.layer_norm(embeddings)  # (batch_size, hidden_dim, seq_len)
        
        embeddings = embeddings.transpose(0, 1)
        
        # Directly feed into Transformer, assuming it's modified to work with (batch_size, hidden_dim, seq_len)
        transformer_output = self.transformer(embeddings, embeddings)  # (batch_size, hidden_dim, seq_len)

        embeddings = embeddings.transpose(0, 1)
        
        # Output layer to project back to embedding_dim
        logits = self.fc_out(transformer_output)  # (batch_size, hidden_dim, seq_len)

        logits = logits.transpose(0, 1).transpose(1, 2)
        
        return logits

    def add_noise(self, token_embeddings, t, schedule):
        noisy_embeddings = token_embeddings.clone()
        noise_level = torch.sqrt(schedule.get_variance(t))

        # Vectorized noise addition for the batch
        noise = torch.normal(mean=0, std=noise_level, size=noisy_embeddings.shape).to(noisy_embeddings.device)
        noisy_embeddings = noisy_embeddings + noise
        return noisy_embeddings
    
    def reverse_diffusion(self, noisy_input, schedule):
        x_t = noisy_input
        device = x_t.device

        for t in reversed(range(self.num_timesteps)):
            # Dynamically add positional encodings
            #x_t = x_t.transpose(1, 2)
            
            # x_t = self.positional_encoding(x_t)

            #x_t = x_t.transpose(1, 2)

            # Predict the noise at timestep t
            predicted_noise = self.forward(x_t)

            # Get schedule parameters
            alpha_t = schedule.get_alpha(t)
            beta_t = schedule.get_beta(t)

            # Compute the mean of x_{t-1}
            mean_x_prev = (x_t - beta_t*predicted_noise)/torch.sqrt(alpha_t)

            # Add stochastic Gaussian noise for intermediate steps
            if t > 0:
                variance = beta_t
                std_dev = torch.sqrt(variance)
                noise = torch.randn_like(x_t, device=device) * std_dev
                x_t = mean_x_prev + noise
            else:
                x_t = mean_x_prev

            # Clamp the outputs to valid range
            x_t = torch.clamp(x_t, min=-1.0, max=1.0)

        return x_t

# Optimized loss function (MSE)
def denoising_loss(predicted_embeddings, clean_embeddings):
    """MSE loss function for continuous embeddings."""
    return nn.MSELoss()(predicted_embeddings,clean_embeddings)

class SymbolicRegressionDataset(Dataset):
    def __init__(self, data, vocab, max_seq_len, noise_schedule):
        self.data = data
        self.vocab = vocab
        self.max_seq_len = max_seq_len
        self.noise_schedule = noise_schedule

    def get_input_embeddings(self, tokens):
        """Convert tokens to their continuous embeddings and pad to max_seq_len."""
        embeddings = torch.stack([torch.tensor(token) for token in tokens])  # Default to 0 if token not in vocab
        padded_embeddings = torch.zeros((self.max_seq_len, embeddings.shape[1]))  # Assuming 2D embeddings
        padded_embeddings[:embeddings.size(0), :] = embeddings  # Pad sequences with zeros
        return padded_embeddings

    def add_noise(self, token_ids, t, schedule):
        """Add noise to embeddings."""
        noisy_embeddings = token_ids.clone()
        noise_level = torch.sqrt(schedule.get_variance(t))

        # Precompute noise for batch and apply it
        noise = torch.normal(mean=0.0, std=noise_level, size=noisy_embeddings.shape).to(noisy_embeddings.device)
        noisy_token_embeddings = noisy_embeddings + noise
        return torch.clamp(noisy_token_embeddings, min=0.0, max=1.0)

    def __getitem__(self, idx):
        data_point = self.data[idx]
        tokens = data_point['tokens']
        token_embeddings = self.get_input_embeddings(tokens)

        t = random.randint(0, self.noise_schedule.timesteps - 1)
        noisy_token_embeddings = self.add_noise(token_embeddings, t, self.noise_schedule)

        # Return noisy and clean embeddings in proper shape
        return noisy_token_embeddings.transpose(0, 1), token_embeddings.transpose(0, 1)

    def __len__(self):
        return len(self.data)

# Optimized training loop
def train_diffusion_model(model, train_loader, val_loader, num_epochs=10, patience_num_epochs=3):
    device = setup_device()
    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',patience=2,factor=0.5)
    schedule = CosineNoiseSchedule(timesteps=1000, device=device)
    
    best_val_loss = float('inf')
    num_epochs_without_improvement = 0
    early_stopping = False
    performance_metrics = {"epoch_list": [], "train_loss_list": [], "val_loss_list": []}

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for noisy_tokens, target_embeddings in train_loader:
            noisy_tokens, target_embeddings = noisy_tokens.to(device), target_embeddings.to(device)
            optimizer.zero_grad()
            pred_embeddings = model(noisy_tokens)
            loss = denoising_loss(pred_embeddings, target_embeddings)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()
        
        train_loss = total_loss / len(train_loader)
        performance_metrics['train_loss_list'].append(train_loss)
        
        # Validation phase with reverse diffusion
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for noisy_tokens, target_embeddings in val_loader:
                noisy_tokens, target_embeddings = noisy_tokens.to(device), target_embeddings.to(device)

                pred_embeddings = model.reverse_diffusion(noisy_tokens, schedule)
                loss = denoising_loss(pred_embeddings, target_embeddings)
                val_loss += loss.item()
        
        val_loss = val_loss / len(val_loader)
        performance_metrics['val_loss_list'].append(val_loss)
        performance_metrics['epoch_list'].append(epoch + 1)

        print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss}, Val Loss: {val_loss}')

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            save_model(model, "best_diffusion_model.pt")
            num_epochs_without_improvement = 0
        else:
            num_epochs_without_improvement += 1

        if num_epochs_without_improvement >= patience_num_epochs:
            print(f"Training stopped early at epoch {epoch + 1}. Best validation loss: {best_val_loss}")
            early_stopping = True
            break

    return model, performance_metrics

In [2]:
# Example of loading and preparing the dataset
dataset = load_dataset('Data/preprocessed_data_with_embeddings.json')
vocab = load_JSON("Data/vocab_embeddings.json")  # Vocabulary is a dictionary of continuous embeddings

MAX_LENGTH = determine_max_seq_len(dataset)  # Determine the max length dynamically

schedule = CosineNoiseSchedule(timesteps=1000,device=setup_device())

# First, perform the split on the raw dataset
train_size = int(0.7 * len(dataset))  # 70% for training
val_size = int(0.15 * len(dataset))  # 15% for validation
test_size = len(dataset) - train_size - val_size  # 15% for testing

# Perform random split
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_dataset_SR = SymbolicRegressionDataset(train_dataset, vocab, MAX_LENGTH, schedule)
val_dataset_SR = SymbolicRegressionDataset(val_dataset, vocab, MAX_LENGTH, schedule)
test_dataset_SR = SymbolicRegressionDataset(test_dataset, vocab, MAX_LENGTH, schedule)

# Create DataLoader objects for each subset
train_loader = DataLoader(train_dataset_SR, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset_SR, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset_SR, batch_size=16, shuffle=True)

# Initialize the model
num_heads = 4  # Number of attention heads, ensure this is a divisor of embedding_dim
embedding_dim = 100  # The embedding dimension is 100 as per your problem
hidden_dim = embedding_dim  # Hidden dimension stays the same for simplicity

# Ensure embedding_dim is divisible by num_heads
if embedding_dim % num_heads != 0:
    raise ValueError(f"embedding_dim ({embedding_dim}) must be divisible by num_heads ({num_heads}).")

model = DiffusionModel(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    num_layers=4,
    num_heads=4,
    num_timesteps=1000,
    pretrained_embeddings=vocab
)

model,performance_metrics_DICT = train_diffusion_model(model,train_loader,val_loader,num_epochs=10,patience_num_epochs=3)

c:\Users\matth\OneDrive - University of Waterloo\Documents\Python Files\Environments\STAT940_Final_Project_VENV\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/10, Train Loss: 0.05390751184452148, Val Loss: 0.4232903603050444
Epoch 2/10, Train Loss: 0.018480337956654175, Val Loss: 0.4228579170174069


KeyboardInterrupt: 

### Plotting Results

In [ ]:
import matplotlib.pyplot as plt

#Visualize the train and validation loss
def plot_train_valid(model_name,performance_metrics_DICT):
    plt.figure();
    plt.plot(performance_metrics_DICT['epoch_list'], performance_metrics_DICT['train_loss_list'], label=f'Train Loss', color='blue', linestyle='--', marker='o');
    plt.plot(performance_metrics_DICT['epoch_list'], performance_metrics_DICT['val_loss_list'], label=f'Validation Loss', color='green', linestyle='-', marker='x');
    plt.title(f'{model_name} Training and Validation Loss');
    plt.xlabel('Epochs');
    plt.ylabel('Loss');
    plt.legend();
    plt.grid();
    plt.xlim(0,max(performance_metrics_DICT['epoch_list'])+1);
    return

In [ ]:
model_name = 'Diffusion Model'

plot_train_valid(model_name,performance_metrics_DICT)

### Prediction

In [ ]:
def decode_embeddings_to_tokens(embeddings, vocab):
    vocab_embeddings = torch.stack([torch.tensor(embed) for embed in vocab.values()])
    
    decoded_tokens = []
    for embedding in embeddings:
        # Compute the distance between the embedding and all vocab embeddings
        distances = torch.norm(embedding - vocab_embeddings, dim=1)
        closest_token_idx = torch.argmin(distances).item()
        closest_token = list(vocab.keys())[closest_token_idx]
        decoded_tokens.append(closest_token)
    
    return decoded_tokens

def evaluate_diffusion_model(model, test_loader, vocab, schedule, device):
    model.eval()  # Set the model to evaluation mode
    total_test_loss = 0.0
    decoded_formulas = []
    actual_formulas = []

    with torch.no_grad():
        for noisy_embeddings, target_embeddings in test_loader:
            # Get the predicted denoised embeddings
            t = random.randint(0, model.num_timesteps - 1)  # Random timestep for diffusion
            pred_embeddings = model.reverse_diffusion(noisy_embeddings, schedule)

            # Calculate the loss (MSE between predicted and target embeddings)
            loss = denoising_loss(pred_embeddings, target_embeddings)
            total_test_loss += loss.item()

            # Now, we need to decode the denoised embeddings back to tokens
            decoded_tokens = decode_embeddings_to_tokens(pred_embeddings, vocab)

            # Convert the decoded tokens to a formula string
            predicted_formula = " ".join(decoded_tokens)
            decoded_formulas.append(predicted_formula)

            # Assuming target embeddings have a corresponding ground truth formula (you can adjust this part)
            actual_formula = decode_embeddings_to_tokens(target_embeddings, vocab)
            actual_formulas.append(" ".join(actual_formula))

    # Calculate average test loss
    avg_test_loss = total_test_loss / len(test_loader)
    return avg_test_loss, decoded_formulas, actual_formulas


In [ ]:
# Example of loading and preparing the dataset
dataset = load_dataset('Data/preprocessed_data_with_embeddings.json')
vocab = load_JSON("Data/vocab_embeddings.json")  # Vocabulary is a dictionary of continuous embeddings

MAX_LENGTH = determine_max_seq_len(dataset)  # Determine the max length dynamically

schedule = CosineNoiseSchedule(timesteps=1000,device=setup_device())

# First, perform the split on the raw dataset
train_size = int(0.7 * len(dataset))  # 70% for training
val_size = int(0.15 * len(dataset))  # 15% for validation
test_size = len(dataset) - train_size - val_size  # 15% for testing

# Perform random split
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Initialize the SymbolicRegressionDataset with the schedule for each subset
train_dataset_SR = SymbolicRegressionDataset(train_dataset, vocab, MAX_LENGTH, schedule)
val_dataset_SR = SymbolicRegressionDataset(val_dataset, vocab, MAX_LENGTH, schedule)
test_dataset_SR = SymbolicRegressionDataset(test_dataset, vocab, MAX_LENGTH, schedule)

# Create DataLoader objects for each subset
train_loader = DataLoader(train_dataset_SR, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset_SR, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset_SR, batch_size=16, shuffle=True)

# Initialize the model
num_heads = 4  # Number of attention heads, ensure this is a divisor of embedding_dim
embedding_dim = 100  # The embedding dimension is 100 as per your problem
hidden_dim = embedding_dim  # Hidden dimension stays the same for simplicity

# Ensure embedding_dim is divisible by num_heads
if embedding_dim % num_heads != 0:
    raise ValueError(f"embedding_dim ({embedding_dim}) must be divisible by num_heads ({num_heads}).")

model = DiffusionModel(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    num_layers=6,
    num_heads=num_heads,
    num_timesteps=1000,
    pretrained_embeddings=vocab
)

model, device = load_model(model, 'Data/best_diffusion_model.pt')
model = model.to(device)

In [ ]:
# Example: Evaluate the model on the test set
test_loss, decoded_formulas, actual_formulas = evaluate_diffusion_model(model, test_loader, vocab, schedule, device)

# Print out the average test loss
print(f"Test Loss: {test_loss}")

# Print out the first few decoded formulas and their corresponding actual formulas
for predicted, actual in zip(decoded_formulas[:5], actual_formulas[:5]):
    print(f"Predicted Formula: {predicted}")
    print(f"Actual Formula: {actual}")
    print("-" * 50)